<a href="https://colab.research.google.com/github/Anyulund/Projects_overview/blob/master/Neural_Networks_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
tf.random.set_random_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
sess = tf.Session(graph = tf.get_default_graph(), config=session_conf)
K.set_session(sess)

TensorFlow version: 1.14.0


In [ ]:
VERSION = "v4"
DATA_PATH = "/content/drive/My Drive/ML Data Playground/"

In [ ]:
train_dataset = pd.read_csv(DATA_PATH + "cleaned_train_data_" + VERSION + ".csv")
test_dataset = pd.read_csv(DATA_PATH + "cleaned_test_data_" + VERSION + ".csv")

In [ ]:
# train_dataset.drop(columns = ["Item_Identifier", "Outlet_Identifier"], axis = 1, inplace = True)
train_labels = train_dataset.pop("Item_Outlet_Sales")
train_dataset.head()

,Item_MRP,Item_Visibility,Item_Weight,Outlet_Years,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Size_Medium,Outlet_Size_High,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Broad_Type_Food,Item_Broad_Type_Non-Consumable
0,1.752511,-0.969852,-0.751014,-0.139681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
1,-1.493696,-0.907063,-1.477653,-1.334228,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0
2,0.009874,-0.956000,1.011839,-0.139681,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0
3,0.661838,-1.281712,1.377308,-0.020226,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
4,-1.403623,-1.281712,-0.830557,1.293777,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1


In [ ]:
test_dataset.head()

,Item_Identifier,Item_MRP,Item_Visibility,Item_Weight,Outlet_Identifier,Outlet_Years,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Size_Medium,Outlet_Size_High,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Broad_Type_Food,Item_Broad_Type_Non-Consumable
0,FDW58,-0.533831,-1.134699,1.710530,OUT049,-0.139681,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
1,FDW14,-0.864708,-0.534917,-0.965996,OUT017,-1.095319,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0
2,NCN55,1.622763,0.653405,0.388391,OUT010,-0.020226,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1
3,FDQ58,0.225966,-0.982657,-1.177753,OUT017,-1.095319,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0
4,FDY38,1.501577,1.023121,0.173409,OUT027,1.532686,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0


In [ ]:
item_identifiers = test_dataset.pop("Item_Identifier")
outlet_identifiers = test_dataset.pop("Outlet_Identifier")

In [ ]:
from keras.layers import Dense, Dropout, ReLU, LeakyReLU

In [ ]:
def build_model(optimizer_func, 
                hidden_nodes_layer1, hidden_nodes_layer2,
                dropout_rate_layer1, dropout_rate_layer2,
                activation_function_layer1, activation_function_layer2):
  """
  Creating a custom function for building an ann model.
  """
  
  model = keras.Sequential()
  model.add(Dense(hidden_nodes_layer1, activation="linear", input_shape=[len(train_dataset.keys())]))
  model.add(activation_function_layer1())
  model.add(Dropout(dropout_rate_layer1, seed = SEED),)
  model.add(Dense(hidden_nodes_layer2, activation="linear"))
  model.add(activation_function_layer2())
  model.add(Dropout(dropout_rate_layer2, seed = SEED),)
  model.add(Dense(1))

  model.compile(loss = "mean_squared_error", 
                optimizer = optimizer_func(), 
                metrics = ["mean_absolute_error", "mean_squared_error"])
  
  return model

In [ ]:
def save_to_csv(model_predictions, filename):
  result = pd.DataFrame({
    "Item_Identifier": item_identifiers,
    "Outlet_Identifier": outlet_identifiers,
    "Item_Outlet_Sales": model_predictions
  })

  result.to_csv(filename, index = False)

In [ ]:
EPOCHS = 35

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

hyperparameters = {
  "hidden_nodes_layer1": np.arange(20, 100, 1),
  "hidden_nodes_layer2": np.arange(20, 100, 1),
  "dropout_rate_layer1": np.arange(0.001, 0.4, 0.01),
  "dropout_rate_layer2": np.arange(0.001, 0.4, 0.01),
  "activation_function_layer1": [ReLU, LeakyReLU],
  "activation_function_layer2": [ReLU, LeakyReLU],
  "optimizer_func": [keras.optimizers.RMSprop]
}

model = KerasRegressor(build_fn = build_model, epochs = EPOCHS)
random_search = RandomizedSearchCV(estimator = model, param_distributions = hyperparameters, cv = 3, n_iter = 15, random_state=SEED)
random_search.fit(train_dataset, train_labels, verbose = 0)

2841/2841 [==============================] - 10s 4ms/step


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f81c2912780>,
                   iid='warn', n_iter=15, n_jobs=None,
                   param_distributions={'activation_function_layer1': [<class 'keras.layers.advanced_activations.ReLU'>,
                                                                       <class 'keras.layers.advanced_activations.LeakyReLU'>],
                                        'activation_function_layer2': [<...
                                        'hidden_nodes_layer2': array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'op

In [ ]:
random_search.best_params_

{'activation_function_layer1': keras.layers.advanced_activations.ReLU,
 'activation_function_layer2': keras.layers.advanced_activations.LeakyReLU,
 'dropout_rate_layer1': 0.12099999999999998,
 'dropout_rate_layer2': 0.001,
 'hidden_nodes_layer1': 58,
 'hidden_nodes_layer2': 72,
 'optimizer_func': keras.optimizers.RMSprop}

In [ ]:
best_estimator_predictions = random_search.best_estimator_.predict(test_dataset).flatten()
save_to_csv(best_estimator_predictions, "ann_tuned_data_" + VERSION + "_predictions.csv")

# Saving the model

In [ ]:
import h5py
from keras.models import model_from_json

In [ ]:
# Saving the model
model_json = random_search.best_estimator_.model.to_json()
with open(DATA_PATH + "model.json", "w") as json_file:
    json_file.write(model_json)

random_search.best_estimator_.model.save_weights(DATA_PATH + "model.h5")

In [ ]:
# # Loading the model
# json_file = open("model.json", "r")
# loaded_model_json = json_file.read()
# json_file.close()

# loaded_model = model_from_json(loaded_model_json)
# loaded_model.load_weights("model.h5")

# # Predicting using the loaded model
# loaded_model_predictions = loaded_model.predict(test_dataset).flatten()

# TODOs
* Try with other data preprocessing versions
* Try with lower number of hidden nodes. For example: 4 hidden nodes in the first layer and then 2 in the second hidden layer
* change to TPU
* https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.49238&showTestData=false&discretize=false&percTrainData=20&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=regression&initZero=false&hideText=false